In [ ]:
import pandas as pd
import numpy as np
from locale import atof, setlocale, LC_NUMERIC, LC_ALL
import matplotlib.pyplot as plt
import matplotlib_inline
import seaborn as sns
import scipy
from scipy.stats import norm
setlocale(LC_ALL, 'fr_FR.UTF-8')

In [ ]:
pd.set_option('display.max_columns', None)

# GetData

In [1]:
import pandas as pd

class GetData:
    """ Read data from csv and load it in a dataframe
    accepted arguments : path to file , separator, chunksize and filter
    option to load csv by filtering on house type
    """

    def __init__(self,path ="../data/valeursfoncieres-2021.txt",sep = "|", chunksize = 100000):
        self.path = path
        self.sep = sep
        self.chunksize = chunksize


    def read_csv(self, filtering_column='Code type local', filter=[1]):
        """ pass option on which column to filter and filter value
        if several filter value, pass the as a list"""
        iter_csv = pd.read_csv(self.path,
                               sep=self.sep,
                               iterator=True,
                               chunksize=self.chunksize,
                               low_memory=False)
        self.df = pd.concat([
            chunk[chunk[filtering_column].isin(filter)] for chunk in iter_csv
        ])
        return self.df

    def enrichissement_coordinates(self,df):
        pass

    def enrichissement_insee(self,df):
        pass

    def loading_data_db (self,df):
        pass


# Preprocessing

Tout l'objectif de cette étape est de tirer le maximum du dataset, en reconstituant ce qui correspond réellement à une transaction. 
Nous avons réalisé cela en regroupant chaque transaction selon 3 clés : Parcelle / date / montant car ceux ci sont répété sur les lignes communes. 
Ensuite nous avons utilisé la methode apply qui pour chaque ligne du dataset aggrégé applique une fonction lambda dans laquelle on inséré une série dont les paramétres étaient la ligne aggrée. cela nous a permis de réaliser des fonctions plus complexes au niveau des aggrégats et de couvrir les différents cas ammenant à la mutltiplication des lignes 
Les cas plus complexes à retrouver sont par exemple la nature culture basée principale en se basant sur la superficie de celle ci (--- piste https://stackoverflow.com/questions/23394476/keep-other-columns-when-doing-groupby 
L'autre cas complexe était de compter le nombre de dépendances différentes ainsi que le nombre de maisons différentes au sein d'une même aggrégation -**ajouter cas ou on a plusieurs dépendances et plusieurs terrains -- trop complexe** : 
* #dépendance - count du terme dépendance valuecount sorted[0]  / nunique type local si nunique nat culture = 1 & len value count ==2 else count terme dependance valuecount sorted[0]   if len_value count ==2 else 0
* #maisons - count terme maison value count sorted [1] / nunqiue type local si nunique nat culture = 1 & len value count ==2 else count terme maison value count if len value_count ==2 else 0 

In [2]:
#from house_prediction_package.data import GetData
import pandas as pd
import numpy as np
from datetime import datetime
from more_itertools import chunked
from scipy import stats

from sklearn.model_selection import train_test_split
#sans doute à supprimer au lancement final du modele
from locale import atof, setlocale, LC_NUMERIC, LC_ALL

setlocale(LC_ALL, 'fr_FR.UTF-8')

class Preprocessing :

    def __init__(self,df) :
        # self.df = get_data().read_csv()
        self.df = df

    def clean_columns(self,
                      columns=[
                          'Code service CH', 'Reference document',
                          '1 Articles CGI', '2 Articles CGI', '3 Articles CGI',
                          '4 Articles CGI', '5 Articles CGI', 'No Volume',
                          'Identifiant local'
                      ]):
        """ drop useless columns
        Customisation of columns to drop must be entered as a list
        """
        # suppression of 100% empty columns - these columns are officially not completed in this db
        self.df = self.df.drop(columns,axis=1)
        # suppression of columns poorly completed
        columns_to_drop = [column for column in self.df.columns if ((self.df[column].isnull().value_counts().sort_index()[0]/self.df.shape[0])*100) < 2 ]
        self.df= self.df.drop(columns_to_drop,axis=1)
        # suppression of nan value on target variable
        self.df= self.df.dropna(subset=['Valeur fonciere'])
        # pre processing avant groupby mais attention sortir valeures foncieres avant de mettre en POO
        ob_columns= self.df.dtypes[self.df.dtypes == 'O'].index
        num_columns = self.df.dtypes[(self.df.dtypes == 'int')
                                     | (self.df.dtypes == 'float')].index
        non_num_col = ['No disposition', 'No voie', 'Code postal', 'Code commune',
       'Prefixe de section', 'No plan','Code type local']
        num_columns = [value for value in num_columns if value not in non_num_col]
        for column in ob_columns :
            self.df[column]=self.df[column].replace(np.nan,'',regex=True)
        #à adapter in v2
        
        self.df[num_columns] = self.df[num_columns].apply(pd.to_numeric,
                                                              errors='coerce')
        
        #drop duplicates
        self.df = self.df.drop_duplicates().reset_index(drop= True)
        # by returning self, we can do method chaining like preprocessing(df).clean_columns().create_identifier()
        return self.df

    def create_identifier(self) :
        """ Create a 'unique' identifier allowing us to group several lines corresponding to a unique transaction
        """
        variables_to_clean = [
            "Code departement", "Code commune", "Prefixe de section",
            "Section", "No plan"
            ]
        size_variables= [2,3,3,2,4]
        for i,j in zip(variables_to_clean,size_variables):
            chunked_data = chunked(self.df[i], 10000, strict=False)
            values = {"Prefixe de section": '000'}
            self.df= self.df.fillna(value=values)
            if i == "Prefixe de section" :
                self.df[i] = self.df[i].apply(str).apply(lambda x: x[:3])
            new_variable = [
                str(value).replace(".","").zfill(j) for sublist in list(chunked_data)
                for value in sublist
            ]
            self.df[f"clean_{i.replace(' ','_').lower()}"] = new_variable
            self.df= self.df.drop([i],axis=1)
        self.df["parcelle_cadastrale"] = self.df[[
            "clean_code_departement", "clean_code_commune", "clean_prefixe_de_section",
            "clean_section", "clean_no_plan"]].apply(lambda x: "".join(x), axis=1)
        self.df["parcelle_cad_section"]=self.df["parcelle_cadastrale"].str[:10]
        self.df = self.df.drop([
            "clean_prefixe_de_section", "clean_section", "clean_no_plan"
        ], axis = 1)
        return self.df

    def aggregate_transactions(self):
        self.df = self.df.groupby(["parcelle_cad_section","Date mutation","Valeur fonciere"], as_index= False).apply(lambda x : pd.Series({
            "B_T_Q" : x["B/T/Q"].max()
            ,"type_de_voie": x["Type de voie"].max()
            ,"voie": x["Voie"].max()
            ,"code_postal": x["Code postal"].max()
            ,"commune": max(x["Commune"])
            ,"clean_code_departement": x["clean_code_departement"].max()
            ,"clean_code_commune": max(x["clean_code_commune"])
           # ,"surface_carrez_lot_1" :  x["Surface Carrez du 1er lot"].sum()/((x["Surface reelle bati"].count()/x["Nature culture"].nunique()))
            ,"Nb_lots": x["Nombre de lots"].max()
            ,"surface_terrain" : ((x["Surface terrain"].sum()/x["Surface reelle bati"].count()) if (int(x["Surface terrain"].nunique()) ==1 and int(x["Nature culture"].nunique()) == 1 )else x["Surface terrain"].sum())
            ,"surface_reelle_bati" : (x["Surface reelle bati"].sum()/(x["Surface reelle bati"].count()/x["Type local"].nunique()) if (int(x["Nature culture"].nunique() > 1)) else x["Surface reelle bati"].sum())
            ,"nb_pieces_principales" : (x["Nombre pieces principales"].sum()/(x["Surface reelle bati"].count()/x["Type local"].nunique()) if int(x["Nature culture"].nunique()) > 1 else x["Nombre pieces principales"].sum())      
            ,"dependance" : x["Type local"].unique()
            ,"main_type_terrain" : x["Nature culture"].max()
            ,"parcelle_cadastrale": x["parcelle_cadastrale"].max()}))
        #drop rows with only dependances transactions as we focus on houses
        self.df = self.df[self.df.dependance.apply(
            lambda x: x.all() != "Dépendance")].reset_index(drop=True)
        self.df["dependance"] = self.df.dependance.apply(sorted, 1)
        self.df[["Dependance",
                 "Maison"]] = pd.DataFrame(self.df.dependance.tolist(),
                                           index=self.df.index)
        self.df["Dependance"] = [1 if value =="Dépendance"else 0 for value in self.df["Dependance"]]
        self.df= self.df.drop(["dependance","Maison"],axis =1)
        return self.df

    # to do : function calling enrichissement from data


    def feature_generation (self):
        # convert the 'Date' column to datetime format
        self.df["month"] = pd.to_datetime(
            self.df["Date mutation"],format="%d/%m/%Y").dt.month
        self.df= self.df.drop(["Date mutation"], axis = 1)
        ## attention à ne faire qu'après avoir enrichi avec variables insee
        dict_type_voie = dict()
        for value in self.df["type_de_voie"].value_counts()[self.df["type_de_voie"].value_counts()<300 ].index.values :
            dict_type_voie[value] = "Autres"
        self.df=self.df.replace({"type_voie" : dict_type_voie})
        self.df["type_de_voie"]= self.df["type_de_voie"].replace(np.nan,'vide')
        return self.df

    def zscore (self) :
        # Calculate the z-score from scratch
        #self.df['Valeur fonciere']= df['Valeur fonciere'].apply(lambda x: atof(x))
        standard_deviation = self.df["Valeur fonciere"].std(ddof=0)
        mean_value = self.df["Valeur fonciere"].mean()
        zscores = [(value - mean_value) / standard_deviation
                for value in self.df["Valeur fonciere"]]
        self.df["zscores"]= zscores
        # absolute value of zscore and if sup x then 1  :
        self.df["outlier"] = [
            1 if (abs(value) > 3) else 0 for value in self.df["zscores"]
        ]
        self.df=self.df[self.df["outlier"] == 0].reset_index(drop=True)
        self.df = self.df.drop(["zscores","outlier"], axis = 1)
        return self.df

    def split_x_y (self):
        columns_model = ["type_de_voie",
            "clean_code_departement",
            "clean_code_commune",
            "code_postal",
            "surface_terrain",
            "surface_reelle_bati", "nb_pieces_principales",
            "main_type_terrain",  "Dependance",
            "month"]
        # Séparation des variables catégorielles et numériques
        categorical_features = [
            "type_de_voie", "clean_code_departement", "clean_code_commune",
            "code_postal", "main_type_terrain", "Dependance", "month"
        ]
        numerical_features = [
            "surface_terrain", "surface_reelle_bati", "nb_pieces_principales"
        ]
        for column in categorical_features:
            self.df[column] = self.df[column].replace(np.nan, "").apply(str)
        X = self.df[columns_model]
        y =self.df["Valeur fonciere"]
        # selection des variables
        X_train, X_test, y_train, y_test = train_test_split(X,
                                                            y,
                                                            test_size=0.33,
                                                            random_state=42)
        return self.df,categorical_features, numerical_features, X_train, X_test, y_train, y_test


# Pipeline 

In [3]:
import numpy as np
import pandas as pd
from sklearn.impute import KNNImputer

from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler


from sklearn.model_selection import cross_val_score
from sklearn.model_selection import learning_curve

from sklearn.linear_model import LinearRegression

import warnings
warnings.filterwarnings('ignore')

from sklearn import pipeline
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer

#from house_prediction_package.preprocessing import Preprocessing
#from house_prediction_package.data import GetData


class Pipeline :

    def __init__(self, df):
        self.df = df
        # self.categorical_features = categorical_features
        # self.numerical_features = numerical_features
        # self.X_train = X_train
        # self.y_train = y_train
        # option 2
        #appeler les méthodes
        self.df, self.categorical_features, self.numerical_features, self.X_train, self.X_test, self.y_train, self.y_test = Preprocessing(
            df).feature_generation().zscore().split_x_y()

    def pipeline(self):
        # création des pipelines de pré-processing pour les variables numériques et catégorielles
        #ajout d'un parametre pour gerer les valeures non connues dans onehotencoder - il les passe à 0(autres options disponibles)
        numerical_pipeline = make_pipeline(KNNImputer(n_neighbors=3), MinMaxScaler())
        categorical_pipeline = make_pipeline(OneHotEncoder(handle_unknown="ignore"))
        preprocessor = make_column_transformer(
            (numerical_pipeline, self.numerical_features),
            (categorical_pipeline, self.categorical_features))
        model = make_pipeline(preprocessor, LinearRegression())
        fitted_model = model.fit(self.X_train, self.y_train)
        return fitted_model, self.X_train, self.y_train,self.X_test, self.y_test


# Zone de tests

## Test model sans aggrégation 

In [ ]:
df = GetData().read_csv()

In [ ]:
df,categorical_features, numerical_features, X_train, X_test, y_train, y_test = Preprocessing(df).split_x_y()

In [ ]:
numerical_pipeline = make_pipeline(KNNImputer(n_neighbors=3), MinMaxScaler())
categorical_pipeline = make_pipeline(OneHotEncoder(handle_unknown="ignore"))
preprocessor = make_column_transformer(
            (numerical_pipeline, numerical_features),
            (categorical_pipeline, categorical_features))
model = make_pipeline(preprocessor, LinearRegression())
fitted_model = model.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

test_y_hat = fitted_model.predict(X_test)
print('Score r²: ', fitted_model.score(X_test, y_test))
print("Mean absolute error: %.2f" % np.mean(np.absolute(test_y_hat - y_test)))
print("Residual  of squares (MSE): %.2f" % np.mean((test_y_hat - y_test)**2))
print("R2-score: %.2f" % r2_score(test_y_hat, y_test))

In [ ]:
import pickle
# save the model to disk
filename = 'house_model_wo_aggregations.sav'
pickle.dump(fitted_model, open(filename, 'wb'))
 
# some time later...
 
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))
#result = loaded_model.score(X_test, Y_test)
#print(result)

## Test model maison avec aggrégation 

In [4]:
df= GetData().read_csv()

In [5]:
df = Preprocessing(df).clean_columns()

In [6]:
df= Preprocessing(df).create_identifier()

In [ ]:
df = Preprocessing(df).aggregate_transactions()

In [ ]:
df.to_csv("aggregatedfile_houses.csv", sep='|', encoding="utf-8") 

In [ ]:
df = Preprocessing(df).feature_generation()

In [ ]:
df = Preprocessing(df).zscore() 

In [ ]:
df,categorical_features, numerical_features, X_train, X_test, y_train, y_test = = Preprocessing(df).split_x_y()

In [ ]:
numerical_pipeline = make_pipeline(KNNImputer(n_neighbors=3), MinMaxScaler())
categorical_pipeline = make_pipeline(OneHotEncoder(handle_unknown="ignore"))
preprocessor = make_column_transformer(
            (numerical_pipeline, numerical_features),
            (categorical_pipeline, categorical_features))
model = make_pipeline(preprocessor, LinearRegression())
fitted_model = model.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import math

test_y_hat = fitted_model.predict(X_test)
print('Score r²: ', fitted_model.score(X_test, y_test))
print("Mean absolute error: %.2f" % np.mean(np.absolute(test_y_hat - y_test)))
print("Residual  of squares (MSE): %.2f" % np.mean((test_y_hat - y_test)**2))
print("R(MSE): %.2f" % math.sqrt(np.mean((test_y_hat - y_test)**2))
print("R2-score: %.2f" % r2_score(test_y_hat, y_test))

In [ ]:
import pickle
# save the model to disk
filename = 'house_model_aggregations.sav'
pickle.dump(fitted_model, open(filename, 'wb'))
 
# some time later...
 
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))
#result = loaded_model.score(X_test, Y_test)
#print(result)

## test model maison/dep with aggregations

In [ ]:
df= GetData().read_csv([1,3])

df = Preprocessing(df).clean_columns()

df= Preprocessing(df).create_identifier()

df = Preprocessing(df).aggregate_transactions()

df = Preprocessing(df).feature_generation()

df = Preprocessing(df).zscore() 

df,categorical_features, numerical_features, X_train, X_test, y_train, y_test = = Preprocessing(df).split_x_y()

numerical_pipeline = make_pipeline(KNNImputer(n_neighbors=3), MinMaxScaler())
categorical_pipeline = make_pipeline(OneHotEncoder(handle_unknown="ignore"))
preprocessor = make_column_transformer(
            (numerical_pipeline, numerical_features),
            (categorical_pipeline, categorical_features))
model = make_pipeline(preprocessor, LinearRegression())
fitted_model = model.fit(X_train, y_train)

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import math

test_y_hat = fitted_model.predict(X_test)
print('Score r²: ', fitted_model.score(X_test, y_test))
print("Mean absolute error: %.2f" % np.mean(np.absolute(test_y_hat - y_test)))
print("Residual  of squares (MSE): %.2f" % np.mean((test_y_hat - y_test)**2))
print("R(MSE): %.2f" % math.sqrt(np.mean((test_y_hat - y_test)**2))
print("R2-score: %.2f" % r2_score(test_y_hat, y_test))

import pickle
# save the model to disk
filename = 'house_dep_model_aggregations.sav'
pickle.dump(fitted_model, open(filename, 'wb'))
 
# some time later...
 
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))
#result = loaded_model.score(X_test, Y_test)
#print(result)

## cas de tests sur aggregations 

In [ ]:
df= GetData().read_csv()


In [ ]:
df = Preprocessing(df).clean_columns()

df= Preprocessing(df).create_identifier()

In [ ]:
df2 = df[0:10000]

In [ ]:
df2.groupby(["parcelle_cad_section","Date mutation","Valeur fonciere"], as_index= False).apply(lambda x : pd.Series({
"surface_terrain" : ((x["Surface terrain"].sum()/x["Surface reelle bati"].count()) if (int(x["Surface terrain"].nunique()) ==1 and int(x["Nature culture"].nunique()) == 1 )else x["Surface terrain"].sum())
    ,"Nature_culture" : x["Nature culture"].max()
    , "su terrain 2": x["Surface terrain"].sum()
    , "suterrainmax": x["Surface terrain"].max()
  #  , "su_bat": x["Surface reelle bati"]
    ,"nat_terrain_unique": x["Nature culture"].nunique()
    , "suterrain_count" : x["Surface terrain"].count()
    ,"suterrain_unique": x["Surface terrain"].nunique()
    ,"su_bat_unique" : x["Surface reelle bati"].nunique()
    ,"su_bat_count" : x["Surface reelle bati"].count()
            
})).tail(40)

In [ ]:
df2.groupby(["parcelle_cad_section","Date mutation","Valeur fonciere"], as_index= False).apply(lambda x : pd.Series({
    "surface_reelle_bati" : (x["Surface reelle bati"].sum()/(x["Surface reelle bati"].count()/x["Type local"].nunique()) if (int(x["Nature culture"].nunique() > 1)) else x["Surface reelle bati"].sum())
 ,"nb_pieces_principales" : (x["Nombre pieces principales"].sum()/(x["Surface reelle bati"].count()/x["Type local"].nunique()) if int(x["Nature culture"].nunique()) > 1 else x["Nombre pieces principales"].sum())      
    ,"nb_piecemax" : x["Nombre pieces principales"].max()
    ,"Nature_culture" : x["Nature culture"].max()
    , "su bat 2": x["Surface reelle bati"].sum()
    , "sumax": x["Surface reelle bati"].max()
  #  , "su_bat": x["Surface reelle bati"]
    , "su_count" : x["Surface reelle bati"].count()
    ,"nat_cul_unique": x["Nature culture"].nunique()
    ,"subatiment_unique": x["Surface reelle bati"].nunique()
            
})).tail(40)

In [ ]:
df_tot= GetData().read_csv([1,3])

df_tot = Preprocessing(df_tot).clean_columns()

df_tot= Preprocessing(df_tot).create_identifier()

In [ ]:
# cas ou type local identique mais nature culture différente: 

df[(df['parcelle_cadastrale']== '01289000AC0176') | (df['parcelle_cadastrale']== '013500000C1248')| (df['parcelle_cadastrale']== '01195000AD0050')]
#actions possibles : 
# meme valeur fonciere 
# meme de surface reelle bati 
#pas d'info sur 1er lot
# pas d info Nombre de lots 
#meme nombre pieces principales 
# différente nature culture (variable texte )
# différente surface terrain 


In [ ]:
df[df['parcelle_cad_section']=='01001000ZH']

In [ ]:
# même maison surface reelle bati identique 
df[(df['parcelle_cadastrale'] == '013500000C1248')]
#actions possibles : 
# meme valeur fonciere 
# meêm type local 
# même surface relle bati
 #pas d'info sur 1er lot
# pas d info Nombre de lots 
# même nombre de pieces principales

# différence nature culture (variable texte )
# surface terrain différente en fonction de la parcelle cadastrale 


In [ ]:
# cas ou 2 maisons , une dépendance et un terrain :
# repérable par section et date commune 

df[df.index.isin([71,72,73,74])]
#actions possibles : 
# meme valeur fonciere 
# code type lcoal différent pr dépendance absent pour terrain
# différence de surface reelle bati  (0 dépendance et nan pour terrain)
#pas d'info sur 1er lot
# pas d info Nombre de lots 
# différence sur nombre de pieces principales

# différence nature culture (variable texte )
# surface terrain différente en fonction de la parcelle cadastrale 
